Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [1]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [2]:
import funciones

In [3]:
#anio = int(input('Ingrese el año: '))
#mes = int(input('Ingrese el mes: '))

anio = 2024
mes = 3

In [4]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [5]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [6]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [7]:
#boletas

___________________________________________________________________________________________

In [8]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [9]:
# Creamos un dataframe con las columnas que se van a utilizar
# Selecciona las columnas que se van a utilizar
bol = boletas[boletas['tp_poll_impute_status'].isin([351821, 351824])]
bol = boletas[boletas['detail_tp_poll_status'].isin([351634, 351635])]
bol = bol[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']]
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = bol['cantidad_anterior'].astype(float)
bol['cantidad_actual'] = bol['cantidad_actual'].astype(float)
bol['precio_anterior'] = bol['precio_anterior'].astype(float)
bol['precio_actual'] = bol['precio_actual'].astype(float)
bol['cantidad_anterior'] = bol.apply(lambda row: row['cantidad_actual'] if row['cantidad_anterior'] == 0 or pd.isnull(row['cantidad_anterior']) else row['cantidad_anterior'], axis=1)
bol['precio_anterior'] = bol.apply(lambda row: row['precio_actual'] if row['precio_anterior'] == 0 or pd.isnull(row['precio_anterior']) else row['precio_anterior'], axis=1)

#bol

In [10]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = funciones.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pon_prod['ponderacion_region'].astype(float)
pon_prod['indice_anterior'] = pon_prod['indice_anterior'].astype(float)
#pon_prod

In [11]:
#pon_prod['ponderacion_region'].dtype
pon_prod['ponderacion_region'].isnull().sum()

0

Calculo del IPC

In [12]:
# Se calcula la variacion de las variedades
variacion = funciones.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]

In [13]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
#relativo

In [14]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]

In [15]:
#promedio ponderado
prom_pon = relativo.groupby(['region', 'cod_prod', 'grupo_nombre']).apply(lambda x: pd.Series({
    'var_ac': (x['relativo'] * x['ponderacion_region']).sum(),
    'sum_pon': x['ponderacion_region'].sum()
})).reset_index()
prom_pon['prom_pon'] = prom_pon['var_ac'] / prom_pon['sum_pon']
prom_pon = pd.merge(prom_pon, relativo[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
                  on=['region', 'cod_prod', 'grupo_nombre'], how='left')
#prom_pon

Índice de precios por región

In [16]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior'] * prom_pon['prom_pon']
ind_prod = funciones.cod_grupos(prom_pon)
ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].dropna()
ind_prod01

,region,cod_prod,ind_prod
0,1,0111101,105.617871
1,1,0111102,99.587372
2,1,0111201,97.573906
3,1,0111202,103.434991
4,1,0111203,101.699437
...,...,...,...
2928,8,1329101,100.000000
2929,8,1329102,100.000000
2930,8,1390901,100.000000
2931,8,1390902,100.000000


In [17]:
#indice subclase
funciones.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,01111,104.134510
1,1,01112,100.403994
2,1,01113,99.544265
3,1,01114,99.411848
4,1,01115,99.922783
...,...,...,...
1192,8,13131,100.928775
1193,8,13132,100.000000
1194,8,13211,100.000000
1195,8,13291,100.000000


In [22]:
#indice clase
funciones.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0111,99.936546
1,1,0112,100.590248
2,1,0113,100.600077
3,1,0114,101.283792
4,1,0115,103.082275
...,...,...,...
667,8,1312,101.371715
668,8,1313,100.751598
669,8,1321,100.000000
670,8,1329,100.000000


In [24]:
#indice grupo
funciones.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,011,100.853089
1,1,012,100.664246
2,1,013,100.000000
3,1,021,99.163694
4,1,023,100.507212
...,...,...,...
318,8,105,100.450904
319,8,111,100.079130
320,8,131,101.237985
321,8,132,100.000000


In [23]:
#indice division
funciones.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,01,100.836369
1,1,02,99.532594
2,1,03,100.452592
3,1,04,100.125832
4,1,05,91.850002
...,...,...,...
95,8,08,100.143699
96,8,09,96.747921
97,8,10,100.257806
98,8,11,100.079130


Índice a nivel republica

In [ ]:
#indice productos


In [25]:
def calculate_replicated_indicators(df, cod_subclase):
    result = df.groupby(cod_subclase).apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum(),
        'count': x[cod_subclase].count()
    })).reset_index()

    result['ind_prod_rep'] = result['sum1'] / result['sum2']
    result.dropna(inplace=True)
    result = result[[cod_subclase, 'ind_prod_rep']]

    return result

In [27]:
calculate_replicated_indicators(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0111101,101.626971
1,0111102,96.478754
2,0111201,97.051348
3,0111202,100.707930
4,0111203,99.832643
...,...,...
432,1390901,100.000000
433,1390902,101.094598
434,1390903,100.000000
435,1390904,100.000000
